In [1]:
import gym

In [2]:
env = gym.make("LunarLander-v2")

In [3]:
state = env.reset()

In [4]:
import torch
torch.cuda.is_available()

True

In [5]:
from replay_buffer2 import ReplayBuffer

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.distributions import Categorical




class PolicyNetwork(nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, state_size, action_size, seed, fc1_units=64, fc2_units=64):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(PolicyNetwork, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)

    def forward(self, state):
        """Build a network that maps state -> action values."""
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        logits = torch.softmax(x, dim=1)
        return logits
    
    def eval_act(self, x):
        output = self.forward(x)
        return output.max(dim=-1)[1].detach().item()


    def sample_action(self, state):
        output = self.forward(state)
        dis = Categorical(output)
        action = dis.sample()
        log_probs  = dis.log_prob(action)
        ent = dis.entropy()
        return dis, action.item(), log_probs, ent

In [69]:
env.action_space.sample()

0

In [7]:
policy = PolicyNetwork(8,4,seed=1)

In [8]:
dis, action, log_probs, ent = policy.sample_action(torch.Tensor(state).unsqueeze(0))

In [86]:
action

tensor([1])

In [12]:
state = env.reset()
score = 0
while True:
    #action = policy.eval_act(torch.Tensor(state).unsqueeze(0))
    dis, action, log_probs, entn = policy.sample_action(torch.Tensor(state).unsqueeze(0))
    print(action)
    ns, reward, done, _ = env.step(action)
    print("reward", reward)
    score += reward
    state = ns
    if done:
        print("score", score)
        break
        

0
reward 0.676696202862189
1
reward -0.015561955448872594
1
reward -0.632726535251037
3
reward 0.9076318452949306
0
reward 0.363860895061606
1
reward -0.6523252908630741
2
reward -4.947293541142488
1
reward -1.1130903089649553
1
reward -1.14151488704769
3
reward 0.39621149250652477
3
reward 0.8624343566458788
2
reward -0.816569833738771
0
reward -0.19214205282995067
2
reward -3.6269904006683076
1
reward -1.2499344782031006
1
reward -1.8766886876876743
3
reward 0.050478085006717494
3
reward 0.1799536182166446
1
reward -1.9547803263039032
0
reward -1.034147849242487
3
reward 0.24100998224739215
2
reward -3.372682634730256
0
reward -0.8661516099932101
2
reward -1.2930186748521237
2
reward -2.159390413152823
3
reward 0.0028633318395361596
2
reward -1.0675761782105042
2
reward -1.1232802831475113
0
reward -0.9306602890704028
2
reward -0.6130478731033293
3
reward -0.0756259065500717
1
reward -2.1665341735927357
3
reward -0.05664998498278237
0
reward -1.2007933135512872
3
reward -0.0854602697

In [94]:
dis.log_prob(action_1)

tensor([-1.4184], grad_fn=<SqueezeBackward1>)

In [18]:
action_0 = torch.Tensor(0)
action_1 = torch.Tensor(1)
action_2 = torch.Tensor(2)
action_3 = torch.Tensor(3)

In [19]:
all_actions = [action_0, action_1, action_2, action_3]

In [36]:
dis.log_prob()

tensor([-4.1067], grad_fn=<SqueezeBackward1>)

In [74]:
env.step(4)

AssertionError: 4 (<class 'int'>) invalid 

In [27]:
from model import QNetwork

In [28]:
q_shift_local = QNetwork(8, 4, 0).to('cuda')


In [29]:
s = obses[0]
a = actions[0]

In [30]:
q_shift_local(s,a)

TypeError: forward() takes 2 positional arguments but 3 were given

In [9]:
memory = ReplayBuffer((8,), (4,), 20000, 'cuda')

In [10]:
memory.load_memory("expert_policy/")

In [11]:
memory.idx = 20000

In [12]:
batch_size = 64

In [15]:
obses, actions, rewards, next_obses, not_dones_no_max = memory.sample(3)

In [16]:
obses.shape, actions.shape

(torch.Size([3, 8]), torch.Size([3, 4]))

In [17]:
for idx in range(obses.shape[0]):
    print(obses[idx])
    

tensor([ 0.0569,  0.5656,  0.0064, -0.2650, -0.0026, -0.0303,  0.0000,  0.0000],
       device='cuda:0')
tensor([-8.9781e-02, -8.0440e-04,  3.7255e-05,  3.4191e-08, -3.4477e-03,
        -1.0984e-05,  0.0000e+00,  1.0000e+00], device='cuda:0')
tensor([ 9.0934e-34,  5.3701e+08,  9.0934e-34,  2.4243e+09, -5.3179e+37,
         2.9230e-39,  2.1212e-39,  3.5020e-06], device='cuda:0')


In [57]:
torch.cat((obses,actions), 1)

tensor([[-0.0677,  0.1933, -0.0776, -0.0181, -0.1381, -0.0505,  0.0000,  0.0000,
          1.0000,  1.0000,  1.0000,  1.0000]], device='cuda:0')

In [37]:
state_action_counter[(obses,actions)]

KeyError: (tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-7.6176e-02,  5.7060e-02,  2.8984e-02, -8.5798e-02,  8.9799e-03,
          8.6264e-02,  0.0000e+00,  0.0000e+00],
        [ 6.0577e-02,  5.0389e-01,  4.7262e-02, -2.5287e-01,  9.4282e-04,
         -9.7643e-04,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 8.0740e-02,  4.5389e-02, -6.4012e-02, -6.9226e-02, -6.1525e-02,
          1.4735e-02,  0.0000e+00,  0.0000e+00],
        [-2.2931e-03,  1.4692e+00, -2.7148e-02,  2.9083e-01, -2.6182e-02,
         -8.1885e-02,  0.0000e+00,  0.0000e+00],
        [-2.1131e-01,  9.0615e-02,  1.5891e-02,  3.7146e-02, -1.6311e-01,
          2.0224e-01,  0.0000e+00,  0.0000e+00],
        [-1.5906e-01,  5.3260e-02, -1.8278e-01, -5.4163e-02, -6.2374e-02,
         -3.9950e-01,  0.0000e+00,  0.0000e+00],
     

In [19]:
actions, rewards

(tensor([[2., 2., 2., 2.]], device='cuda:0'),
 tensor([[0.9621]], device='cuda:0'))

In [44]:
state_action_counter = {}

In [62]:
for i in range(1000):
    obses, actions, rewards, next_obses, not_dones_no_max = memory.sample(64)
    for s, a in zip(obses, actions):
        s = s.unsqueeze(0)
        a = a.unsqueeze(0)
        z = torch.cat((s,a), 1)
        if z in state_action_counter:
            state_action_counter[z] += 1
        else:
            state_action_counter.update({z:1})
        
    

In [64]:
  obses, actions, rewards, next_obses, not_dones_no_max = memory.sample(64)

In [67]:
obses.shape

torch.Size([64, 8])

In [75]:
for i in range(100):
    obses, actions, rewards, next_obses, not_dones_no_max = memory.sample(64)
    obses1, actions1, rewards1, next_obses1, not_dones_no_max1 = memory.sample(64)
    for idx1 in range(obses.shape[0]):
        for idx2 in range(obses.shape[0]):
            #print(obses[idx1])
            if torch.equal(obses[idx1],obses1[idx2]):
                print("new")
                print(obses[idx1])
                print(obses1[idx2])

new
tensor([-0.2527, -0.0108,  0.0096,  0.0386,  0.0510,  0.1248,  1.0000,  1.0000],
       device='cuda:0')
tensor([-0.2527, -0.0108,  0.0096,  0.0386,  0.0510,  0.1248,  1.0000,  1.0000],
       device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tens

tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.],

tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.],

new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 

new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 

...

tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.],

new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 

tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.],

new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 

tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
new
tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0.],

In [63]:
len(state_action_counter)

128000

In [42]:
for v  in state_action_counter.values():
    if v ==2:
        print(v)

In [22]:
test = dict()

In [28]:
test.update({('a','b'):1})

In [29]:
test

{'a': 2, ('a', 'b'): 1}

In [30]:
if ('a','b') in test:
    test[('a','b')] +=1

In [31]:
test

{'a': 2, ('a', 'b'): 2}

In [43]:
for i in state_action_counter:
    print(i)
    break

(tensor([-0.0342,  0.0147, -0.0259, -0.0204,  0.0011, -0.0210,  0.0000,  0.0000],
       device='cuda:0'), tensor([2., 2., 2., 2.], device='cuda:0'))


In [83]:
obses, actions, rewards, next_obses, not_dones_no_max = memory.sample(1)

In [117]:
state_action_counter = {}

In [99]:
s = s.detach().cpu().numpy()

In [100]:
s

array([[ 0.12909555,  0.08128346, -0.00970735, -0.07120464,  0.02656468,
        -0.0008317 ,  0.        ,  0.        ]], dtype=float32)

In [148]:
obses, actions, rewards, next_obses, not_dones_no_max = memory.sample(64)
for s, a in zip(obses, actions):
    s = s.detach().cpu().numpy()
    a = a.detach().cpu().numpy()
    z = tuple((s.tobytes(),a.tobytes()))
    if z in state_action_counter:
        state_action_counter[z] += 1
    else:
        state_action_counter.update({z:1})

In [149]:
len(state_action_counter)

1215

In [150]:
state_action_counter

{(b'\xa01\x04>\xf1w\xa6=\x97\x0b\x1f\xbc\xbd\xd3\x91\xbd*\x9e\xd9<[\x06Z\xba\x00\x00\x00\x00\x00\x00\x00\x00',
  b'\x00\x00\x00@\x00\x00\x00@\x00\x00\x00@\x00\x00\x00@'): 2,
 (b'\xc610\xbeae\x9c\xbc\x87\xe9\t\xbb__s\xb9"\\\x0b>\xb3\x87@:\x00\x00\x80?\x00\x00\x80?',
  b'\x00\x00@@\x00\x00@@\x00\x00@@\x00\x00@@'): 1,
 (b'\xc6yt\xbe\xf6k\x1e=\x86U\xf0=w\xae\xb1\xbce\x93\x1b>\xcb\xa0\r=\x00\x00\x00\x00\x00\x00\x00\x00',
  b'\x00\x00\x00@\x00\x00\x00@\x00\x00\x00@\x00\x00\x00@'): 1,
 (b'\x80\xb6\x06>m\xe3o>[\x84\xe8<N\xf1\\\xbd9\xe6{==\x80|=\x00\x00\x00\x00\x00\x00\x00\x00',
  b'\x00\x00@@\x00\x00@@\x00\x00@@\x00\x00@@'): 1,
 (b'\x80\xbe}=\xef\x83*>R\xa0\xea\xbc\xecI$\xbd\xc9\x04\xa3\xbc0e\xf0\xbc\x00\x00\x00\x00\x00\x00\x00\x00',
  b'\x00\x00\x00@\x00\x00\x00@\x00\x00\x00@\x00\x00\x00@'): 1,
 (b'z&>>\xbfNz>1\x17\xcc<\x93\xb6&\xbd\xfb\xfa\x88=\xa9}\xa8\xbc\x00\x00\x00\x00\x00\x00\x00\x00',
  b'\x00\x00\x00@\x00\x00\x00@\x00\x00\x00@\x00\x00\x00@'): 1,
 (b'\x80\x92Y>1\x1e\x0c?\xd8\xcbR\xbe\x

In [96]:
z.

tensor([[ 1.2910e-01,  8.1283e-02, -9.7074e-03, -7.1205e-02,  2.6565e-02,
         -8.3170e-04,  0.0000e+00,  0.0000e+00,  2.0000e+00,  2.0000e+00,
          2.0000e+00,  2.0000e+00]], device='cuda:0')

In [94]:
if z in state_action_counter:
    state_action_counter[z] +=1
    print("add")

add


In [95]:
state_action_counter

{tensor([[ 1.2910e-01,  8.1283e-02, -9.7074e-03, -7.1205e-02,  2.6565e-02,
          -8.3170e-04,  0.0000e+00,  0.0000e+00,  2.0000e+00,  2.0000e+00,
           2.0000e+00,  2.0000e+00]], device='cuda:0'): 1,
 tensor([[ 1.2910e-01,  8.1283e-02, -9.7074e-03, -7.1205e-02,  2.6565e-02,
          -8.3170e-04,  0.0000e+00,  0.0000e+00,  2.0000e+00,  2.0000e+00,
           2.0000e+00,  2.0000e+00]], device='cuda:0'): 2,
 tensor([[ 1.2910e-01,  8.1283e-02, -9.7074e-03, -7.1205e-02,  2.6565e-02,
          -8.3170e-04,  0.0000e+00,  0.0000e+00,  2.0000e+00,  2.0000e+00,
           2.0000e+00,  2.0000e+00]], device='cuda:0'): 2}